In [3]:
import os
import glob

import rasterio

In [4]:
from osgeo import gdal

In [ ]:
def compare_spatial_reference(output_raster, original_dtm):
    with rasterio.open(output_raster) as out_src, rasterio.open(original_dtm) as orig_src:
        print("Output Raster:")
        print(f"CRS: {out_src.crs}")
        print(f"Transform: {out_src.transform}")
        print(f"Bounds: {out_src.bounds}")
        print("\nOriginal DTM:")
        print(f"CRS: {orig_src.crs}")
        print(f"Transform: {orig_src.transform}")
        print(f"Bounds: {orig_src.bounds}")
        
        if out_src.crs == orig_src.crs:
            print("\nCRS matches between output and original.")
        else:
            print("\nWarning: CRS mismatch between output and original.")
        
        if out_src.transform.almost_equals(orig_src.transform):
            print("Transform matches between output and original.")
        else:
            print("Warning: Transform mismatch between output and original.")



In [21]:
def create_vrt_and_convert_to_cog(input_folders, output_folder, test_mode=False):
    os.makedirs(output_folder, exist_ok=True)
    vrt_path = os.path.join(output_folder, "merged.vrt")
    output_cog_path = os.path.join(output_folder, "final_output_cog.tif")
    
    # Collect all input files from the list of input folders
    input_files = []
    for folder in input_folders:
        input_files.extend(sorted(glob.glob(os.path.join(folder, "**", "*.tif"), recursive=True)))

    if test_mode:
        input_files = input_files[:10]  # Process only the first 10 files in test mode

    # Create a VRT from the input files
    vrt = gdal.BuildVRT(vrt_path, input_files, options=gdal.BuildVRTOptions(resampleAlg='nearest', addAlpha=False))
    
    # Get the CRS from the VRT
    src_crs = vrt.GetProjection()
    vrt = None  # Close the VRT to free resources

    print("Setting GDAL configuration options...")
    gdal.SetConfigOption('GDAL_CACHEMAX', '4096')
    gdal.SetConfigOption('GDAL_NUM_THREADS', 'ALL_CPUS')
    gdal.SetConfigOption('GDAL_MEMORY_LIMIT', '4096')
    
    
    # Define translation options for creating a COG
    translate_options = gdal.TranslateOptions(
        format='COG',
        creationOptions=[
            "COMPRESS=LZW", "PREDICTOR=3", "BIGTIFF=YES",
            "BLOCKSIZE=512", "OVERVIEW_RESAMPLING=NEAREST", "NUM_THREADS=ALL_CPUS"
        ],
        outputSRS=src_crs  # Set the output SRS to match the source
    )
    
    print("Translating VRT to COG...")
    gdal.Translate(output_cog_path, vrt_path, options=translate_options)
    os.remove(vrt_path)  # Remove the VRT after translation

    # Verify the CRS of the output COG
    output_ds = gdal.Open(output_cog_path)
    output_crs = output_ds.GetProjection()
    output_ds = None

    if src_crs == output_crs:
        print("CRS preserved successfully.")
    else:
        print("Warning: CRS may not have been preserved.")

    return output_cog_path

In [19]:
# Example usage
input_folder = '/Users/shuyang/Data/DTM/Sudbury/Sudbury-DTM-11/DTM'
output_folder = '//Users/shuyang/Data/DTM/Sudbury/Sudbury-DTM-11'

input_folder ='/Volumes/WD Green/Data/DTM/Sudbury/Sudbury-DTM-11/DTM'
output_folder = '/Volumes/WD Green/Data/DTM/Sudbury/Sudbury-DTM-11'

## Test

In [24]:
# Example usage
input_folders = ['/Volumes/WD Green/Data/DTM/Sudbury/Sudbury-DTM-11/DTM', '/Volumes/WD Green/Data/DTM/Sudbury/Sudbury-DTM-12']
output_folder = '/Volumes/WD Green/Data/DTM/Sudbury/Sudbury-DTM-11'

# Call the modified function
output_cog = create_vrt_and_convert_to_cog(input_folders, output_folder, True)

if output_cog:
    # Assuming the first folder in input_folders is a representative original DTM
    original_dtm = os.path.join(input_folders[0], os.listdir(input_folders[0])[0])  # Use the first folder
    compare_spatial_reference(output_cog, original_dtm)
else:
    print("COG creation failed, no comparison performed.")

Setting GDAL configuration options...
Translating VRT to COG...
Output Raster:
CRS: EPSG:2958
Transform: | 0.50, 0.00, 484000.00|
| 0.00,-0.50, 5175000.00|
| 0.00, 0.00, 1.00|
Bounds: BoundingBox(left=484000.0, bottom=5165000.0, right=485000.0, top=5175000.0)

Original DTM:
CRS: EPSG:2958
Transform: | 0.50, 0.00, 493000.00|
| 0.00,-0.50, 5172000.00|
| 0.00, 0.00, 1.00|
Bounds: BoundingBox(left=493000.0, bottom=5171000.0, right=494000.0, top=5172000.0)

CRS matches between output and original.


## RUN

In [25]:
input_folders = ['/Volumes/WD Green/Data/DTM/Sudbury/Sudbury-DTM-11/DTM','/Volumes/WD Green/Data/DTM/Sudbury/Sudbury-DTM-12/DTM']  # Updated to a list
output_folder = '/Volumes/WD Green/Data/DTM/Sudbury'
create_vrt_and_convert_to_cog(input_folders, output_folder, False)

Setting GDAL configuration options...
Translating VRT to COG...


'/Volumes/WD Green/Data/DTM/Sudbury/final_output_cog.tif'

In [26]:
input_folders = ['/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-02',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-03',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-04',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-05',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-06',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-07',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-08',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-09',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-10',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-11',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-12',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-13',
                 '/Volumes/Expansion 1/DTM_UNZIP/Sudbury-DTM-14'
                 ]

output_folder = '/Volumes/Expansion 1/DTM_COG'
create_vrt_and_convert_to_cog(input_folders, output_folder, False)


Setting GDAL configuration options...
Translating VRT to COG...


In [ ]:
def legcy_merge_and_convert_to_cog(input_folder, output_folder, test_mode=False):
    os.makedirs(output_folder, exist_ok=True)
    intermediate_file = os.path.join(output_folder, "merged.tif")
    output_cog_path = os.path.join(output_folder, "final_output_cog.tif")
    input_files = sorted(glob.glob(os.path.join(input_folder, "**", "*.tif"), recursive=True))

    if test_mode:
        input_files = input_files[:10]  # Process only the first 10 files in test mode

    # Get the CRS from the first input file
    with rasterio.open(input_files[0]) as src:
        src_crs = src.crs

    print("Merging input files...")
    gdal.Warp(intermediate_file, input_files, options=gdal.WarpOptions(srcSRS=src_crs, dstSRS=src_crs))

    intermediate_ds = gdal.Open(intermediate_file)
    if intermediate_ds is None:
        print("Error: Could not open intermediate file.")
        return None

    print(f"Source CRS: {src_crs}")

    gdal.SetConfigOption('GDAL_CACHEMAX', '1024')
    gdal.SetConfigOption('GDAL_NUM_THREADS', 'ALL_CPUS')
    
    translate_options = gdal.TranslateOptions(
        format='COG',
        creationOptions=[
            "COMPRESS=LZW", "PREDICTOR=2", "BIGTIFF=YES",
            "BLOCKSIZE=512", "OVERVIEW_RESAMPLING=AVERAGE", "NUM_THREADS=ALL_CPUS"
        ],
        outputSRS=src_crs
    )
    
    print("Converting to COG...")
    gdal.Translate(output_cog_path, intermediate_file, options=translate_options)
    os.remove(intermediate_file)

    output_ds = gdal.Open(output_cog_path)
    if output_ds is None:
        print("Error: Could not open output COG file.")
        return None

    output_crs = output_ds.GetProjection()
    output_ds = None

    if src_crs.to_string() == output_crs:
        print("CRS preserved successfully.")
    else:
        print("Warning: CRS may not have been preserved.")
        print(f"Source CRS: {src_crs}")
        print(f"Output CRS: {output_crs}")

    return output_cog_path